In [1]:
import pandas as pd

df = pd.read_excel("../data/runningluc.xlsx", engine="openpyxl")
df.head()

,Distance_done_m,Distance_remaining_m,Target_pace_min_per_km,Actual_pace_min_per_km,Interval_time_min,Cumulative_time_min,Heart_rate_bpm,Address
0,200,10800,5.67,5.52,1.10,1.10,152,Elephant & Castle
1,400,10600,5.67,5.78,1.16,2.26,151,Elephant & Castle
2,600,10400,5.67,5.61,1.12,3.38,151,New Kent Rd
3,800,10200,5.67,5.17,1.03,4.41,152,New Kent Rd
4,1000,10000,5.67,5.48,1.10,5.51,152,Great Dover St


In [2]:
SYSTEM_PROMPT = """You are an enthusiastic yet concise running coach.
Your role is to deliver a real-time update in 1–2 sentences (≤ 35 words).
Speak to the runner in the second person singular. Never invent numbers.
"""

USER_TEMPLATE = """
Current data
------------
Address              : {Address}
Distance covered     : {Distance_done_km:.1f} km
Distance remaining   : {Distance_remaining_km:.1f} km
Current pace         : {Actual_pace} min/km
Target pace          : {Target_pace} min/km
Heart rate           : {Heart_rate_bpm} bpm
"""


def build_prompt(row):
    return SYSTEM_PROMPT + USER_TEMPLATE.format(
        Address=row["Address"],
        Distance_done_km=row["Distance_done_m"]/1000,
        Distance_remaining_km=row["Distance_remaining_m"]/1000,
        Actual_pace=row["Actual_pace_min_per_km"],
        Target_pace=row["Target_pace_min_per_km"],
        Heart_rate_bpm=row["Heart_rate_bpm"],
    )

In [ ]:
import requests, time, json

def ask_ollama(prompt, model="phi3:mini"):
    t0 = time.time()
    resp = requests.post(
        "http://localhost:11434/api/generate",
        json={"model": model, "prompt": prompt},
        timeout=60,
    )
    resp.raise_for_status()                   
    data = resp.json()                      
    return data["response"].strip(), round(time.time() - t0, 2)

sample_row = df.iloc[0]
prompt = build_prompt(sample_row)
answer, latency = ask_ollama(prompt)
print(answer, f"\n(latence : {latency}s)")

HTTPError: 404 Client Error: Not Found for url: http://localhost:11434/api/generate

In [ ]:
from tqdm import tqdm
for _, row in df.iloc[::5].iterrows():      # 1 km sur 5 pour commencer
    msg, _ = ask_ollama(build_prompt(row))
    print(f"[{row['Distance_done_m']/1000:.1f} km] {msg}")